<a href="https://colab.research.google.com/github/Dinusha-hue/Deep_Learning_Lab03/blob/main/DL_Lab_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x

!unzip weights.zip

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Archive:  weights.zip
   creating: weights/
  inflating: weights/bn1_b.csv       
  inflating: weights/bn1_m.csv       
  inflating: weights/bn1_v.csv       
  inflating: weights/bn1_w.csv       
  inflating: weights/bn2_b.csv       
  inflating: weights/bn2_m.csv       
  inflating: weights/bn2_v.csv       
  inflating: weights/bn2_w.csv       
  inflating: weights/bn3_b.csv       
  inflating: weights/bn3_m.csv       
  inflating: weights/bn3_v.csv       
  inflating: weights/bn3_w.csv       
  inflating: weights/conv1_b.csv     
  inflating: weights/conv1_w.csv     
  inflating: weights/conv2_b.csv     
  inflating: weights/conv2_w.csv     
  inflating: weights/conv3_b.csv     
  inflating: weights/conv3_w.csv     
  inflating: weights/dense_b.csv     
  inflating: weights/dense_w.csv     
  inflating: weights/inception_3a_1x1_bn_b.csv  
  inflating: weights/inception_3a_1x1_bn_m.csv  
  inflating: weights/incept

In [2]:
# !pip install fr_utils

In [3]:
!pip install --upgrade tensorflow


In [4]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
# from keras.layers.normalization import BatchNormalization
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
# from keras.layers.merge import Concatenate
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
# from keras.engine.topology import Layer
from tensorflow.keras.layers import Layer, InputSpec
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

import tensorflow
print(tensorflow.__version__)

# np.set_printoptions(threshold=np.nan)

2.17.0


In [5]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96))

In [6]:
print("Total Params:", FRmodel.count_params())

Total Params: 3743280


In [7]:
def triplet_loss(y_true, y_pred, alpha = 0.2):
    """
    Implementation of the triplet loss as defined by formula (3)

    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor images, of shape (None, 128)
            positive -- the encodings for the positive images, of shape (None, 128)
            negative -- the encodings for the negative images, of shape (None, 128)

    Returns:
    loss -- real number, value of the loss
    """

    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]

    ### START CODE HERE ### (≈ 4 lines)
    # Step 1: Compute the (encoding) distance between the anchor and the positive
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)))
    # Step 2: Compute the (encoding) distance between the anchor and the negative
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)))
    # Step 3: subtract the two previous distances and add alpha.
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)
    # Step 4: Take the maximum of basic_loss and 0.0. Sum over the training examples.
    loss = tf.maximum(tf.reduce_mean(basic_loss), 0.0)
    ### END CODE HERE ###

    return loss

In [8]:
# with tf.Session() as test:
with tf.compat.v1.Session() as test:
#    tf.set_random_seed(1)
    tf.random.set_seed(1)
    y_true = (None, None, None)
    y_pred = (tf.random.normal([3, 128], mean=6, stddev=0.1, seed = 1),
              tf.random.normal([3, 128], mean=1, stddev=1, seed = 1),
              tf.random.normal([3, 128], mean=3, stddev=4, seed = 1))
    loss = triplet_loss(y_true, y_pred)

    print("loss = " + str(loss.eval()))

loss = 350.02734


In [9]:
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

In [18]:
database = {}
database["danielle"] = img_to_encoding("images/danielle.png", FRmodel)
database["younes"] = img_to_encoding("images/younes.jpg", FRmodel)
database["tian"] = img_to_encoding("images/tian.jpg", FRmodel)
database["andrew"] = img_to_encoding("images/andrew.jpg", FRmodel)
database["kian"] = img_to_encoding("images/kian.jpg", FRmodel)
database["dan"] = img_to_encoding("images/dan.jpg", FRmodel)
database["sebastiano"] = img_to_encoding("images/sebastiano.jpg", FRmodel)
database["bertrand"] = img_to_encoding("images/bertrand.jpg", FRmodel)
database["kevin"] = img_to_encoding("images/kevin.jpg", FRmodel)
database["felix"] = img_to_encoding("images/felix.jpg", FRmodel)
database["benoit"] = img_to_encoding("images/benoit.jpg", FRmodel)
database["arnaud"] = img_to_encoding("images/arnaud.jpg", FRmodel)
database["rayn"] = img_to_encoding("images/rayn.jpg", FRmodel)

In [27]:
def verify(image_path, identity, database, model):

    encoding = img_to_encoding(image_path, model)

    dist = np.linalg.norm(encoding-database[identity])

    if dist < 0.7:
        print("It's " + str(identity) + ", welcome home!")
        door_open = True
    else:
        print("It's not " + str(identity) + ", please go away")
        door_open = False

    return dist, door_open

In [13]:
verify("images/camera_0.jpg", "younes", database, FRmodel)

It's younes, welcome home!


(0.6671399, True)

In [14]:
verify("images/camera_2.jpg", "kian", database, FRmodel)

It's not kian, please go away


(0.8586889, False)

In [25]:
def who_is_it(image_path, database, model):

    encoding = img_to_encoding(image_path, model)

    min_dist = 100

    output_data = {}

    for (name, db_enc) in database.items():

        # Compute L2 distance between the target "encoding" and the current "emb" from the database. (≈ 1 line)
        dist = np.linalg.norm(encoding-db_enc)

        #save L2 distances to interpret results
        output_data[f'{name}'] = dist

        # If this distance is less than the min_dist, then set min_dist to dist, and identity to name. (≈ 3 lines)
        if dist < min_dist:
            min_dist = dist
            identity = name

    ### END CODE HERE ###

    if min_dist > 0.7:
        print("Not in the database.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))

    return min_dist, identity, output_data

In [30]:
output = who_is_it("images/camera_0.jpg", database, FRmodel)

it's younes, the distance is 0.6671399


In [17]:
output[2]

{'danielle': 1.2107376,
 'younes': 0.6671399,
 'tian': 1.221301,
 'andrew': 0.9958904,
 'kian': 0.8566895,
 'dan': 0.7684447,
 'sebastiano': 0.854448,
 'bertrand': 0.7690577,
 'kevin': 0.8715994,
 'felix': 1.2063179,
 'benoit': 0.8959498,
 'arnaud': 0.8346038}

In [31]:
verify("images/camera_6.jpg", "rayn", database, FRmodel)

It's not rayn, please go away


(0.9950889, False)

In [29]:
output = who_is_it("images/camera_6.jpg", database, FRmodel)

Not in the database.
